In [11]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

# Load the dataset directly from the URL
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
spacex_df = pd.read_csv(url)

# Extract unique launch sites from the DataFrame
launch_sites = spacex_df['Launch Site'].unique()

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the app
app.layout = html.Div([
    html.H1("SpaceX Launch Data Dashboard", style={'text-align': 'center'}),

    # Dropdown for selecting launch site
    dcc.Dropdown(
        id='site-dropdown',  # Unique ID for the dropdown component
        options=[  # List of dropdown options
            {'label': 'All Sites', 'value': 'ALL'},  # All Sites option
            *[{'label': site, 'value': site} for site in launch_sites]  # Options for each launch site
        ],
        value='ALL',  # Default value is 'ALL'
        placeholder="Select a Launch Site here",  # Placeholder text when no site is selected
        searchable=True,  # Allows the dropdown to be searchable
        style={'width': '50%'}  # Optional: Adjust the width of the dropdown
    ),

    # Pie chart to show success vs failure count
    html.Div(dcc.Graph(id='success-pie-chart')),  # Pie chart output here
    html.Br(),

    # Placeholder for the output (to show the result after selecting a site)
    html.Div(id='site-output'),

    # Payload Range Slider
    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=spacex_df['Payload Mass (kg)'].min(),
        max=spacex_df['Payload Mass (kg)'].max(),
        step=100,
        marks={i: f'{i}' for i in range(int(spacex_df['Payload Mass (kg)'].min()), int(spacex_df['Payload Mass (kg)'].max()) + 1, 1000)},
        value=[spacex_df['Payload Mass (kg)'].min(), spacex_df['Payload Mass (kg)'].max()]
    ),
    html.Div(dcc.Graph(id='success-payload-scatter-chart'))  # Scatter plot output here
])

# Callback to update the pie chart and text output based on dropdown selection
@app.callback(
    [Output('success-pie-chart', 'figure'),
     Output('site-output', 'children')],
    [Input('site-dropdown', 'value')]
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        # Data for all sites: Count the successful launches (class=1) by each site
        site_success_counts = spacex_df[spacex_df['class'] == 1]['Launch Site'].value_counts()
        fig = px.pie(
            names=site_success_counts.index,  # Launch sites
            values=site_success_counts.values,  # Successful launch counts per site
            title='Total Successful Launches by Site'
        )
        return fig, html.Div("Displaying total successful launches by site for all launch sites.")
    else:
        # Data for the selected site (Success vs Failure)
        site_data = spacex_df[spacex_df['Launch Site'] == selected_site]
        success_count = site_data['class'].value_counts()
        fig = px.pie(
            names=['Failure', 'Success'], 
            values=[success_count.get(0, 0), success_count.get(1, 0)],
            title=f'Success vs Failure Launches for {selected_site}'
        )
        return fig, html.Div([
            html.H4(f"Launch Site: {selected_site}"),
            html.P(f"Success Count (class=1): {success_count.get(1, 0)}"),
            html.P(f"Failure Count (class=0): {success_count.get(0, 0)}")
        ])

# Callback to update the scatter plot based on selected payload range
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('payload-slider', 'value')]
)
def update_scatter_plot(payload_range):
    min_payload, max_payload = payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= min_payload) & (spacex_df['Payload Mass (kg)'] <= max_payload)]
    
    fig = px.scatter(
        filtered_df, 
        x='Payload Mass (kg)', 
        y='class', 
        color='Launch Site',
        title=f'Success vs Payload Mass for Payload Range {min_payload} - {max_payload} kg'
    )
    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8051)
